In [25]:
# import required modules, packages
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D, Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.svm import SVC
%matplotlib inline

# import datasets, classifiers, performance metrics
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.6.0


In [26]:
# accessing the dataset
from PIL import Image, ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

p = Path("sample_Signature/")
dirs = p.glob("*")
labels_dict = {'forge':0, 'genuin':1}

image_data = []
labels = []

for folder_dir in dirs:
    label = str(folder_dir).split("\\")[-1][:-1]

    for img_path in folder_dir.glob("*.png"):
        # reading/opens each image
        img = Image.open(img_path)
        if 'L' in img.getbands():  # image is black-and-white, colorize it
            img = ImageOps.colorize(img, black="blue", white="white")
        # resizing each image
        img_new = img.resize((200,200))
        # convert each image to array using numpy
        img_array = np.asarray(img_new)
        # append each image array to image_data list
        image_data.append(img_array)
        # match the label to corresponding image
        labels.append(labels_dict[label])

        # image color inversion
        from skimage import util
        import numpy as np
        color_inversion_img = util.invert(img_array)
        # alternate method with numpy
        #color_inversion_img_w_np = np.invert(img)
        img_invert_array = np.asarray(color_inversion_img)
        # append each image array to image_data list
        image_data.append(img_invert_array)
        # match the label to corresponding image
        labels.append(labels_dict[label])

In [27]:
image_data = np.array(image_data, dtype = 'float32')/255.0
labels = np.array(labels)

print(image_data.shape, labels.shape)

(600, 200, 200, 3) (600,)


In [28]:
import random 
combined = list(zip(image_data, labels))
random.shuffle(combined)

image_data[:], labels[:] = zip(*combined)

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(200,200,3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [51]:
image_train, image_test, label_train, label_test = train_test_split(
    image_data, labels, test_size =0.8, shuffle = False)

In [52]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [53]:
model.fit(image_train, label_train, epochs=15)

Epoch 1/15
4/4 [==============================] - 0s 43ms/step - loss: 3.6127 - accuracy: 0.7250
Epoch 2/15
4/4 [==============================] - 0s 42ms/step - loss: 2.9143 - accuracy: 0.6500
Epoch 3/15
4/4 [==============================] - 0s 44ms/step - loss: 1.8188 - accuracy: 0.6750
Epoch 4/15
4/4 [==============================] - 0s 43ms/step - loss: 0.6974 - accuracy: 0.7333
Epoch 5/15
4/4 [==============================] - 0s 43ms/step - loss: 1.2273 - accuracy: 0.6417
Epoch 6/15
4/4 [==============================] - 0s 43ms/step - loss: 0.7374 - accuracy: 0.7833
Epoch 7/15
4/4 [==============================] - 0s 45ms/step - loss: 0.4943 - accuracy: 0.7417
Epoch 8/15
4/4 [==============================] - 0s 43ms/step - loss: 0.4253 - accuracy: 0.8500
Epoch 9/15
4/4 [==============================] - 0s 42ms/step - loss: 0.5349 - accuracy: 0.7250
Epoch 10/15
4/4 [==============================] - 0s 43ms/step - loss: 0.7152 - accuracy: 0.7333
Epoch 11/15
4/4 [============

In [54]:
test_loss, test_acc = model.evaluate(image_test, label_test, verbose=10)

print('\nTest accuracy:', test_acc)


Test accuracy: 0.5562499761581421


In [55]:
# check if a model has already been saved and if it hasn't, save it to the disk
# this is basically the serialization
'''import os.path
if os.path.isfile('models/Real_Forged_signatures.h5') is False:
    model.save('models/Real_Forged_signatures.h5')
    '''
json_string = model.to_json()

In [56]:
import json

# writing to json file
with open("2021-09-10-keras.json", "w") as my_file:
    json_object = json.dump(json_string, my_file)

# previewing content
json_str = json.dumps(json_string)
print(json_str)

"{\"class_name\": \"Sequential\", \"config\": {\"name\": \"sequential_3\", \"layers\": [{\"class_name\": \"InputLayer\", \"config\": {\"batch_input_shape\": [null, 200, 200, 3], \"dtype\": \"float32\", \"sparse\": false, \"ragged\": false, \"name\": \"flatten_3_input\"}}, {\"class_name\": \"Flatten\", \"config\": {\"name\": \"flatten_3\", \"trainable\": true, \"batch_input_shape\": [null, 200, 200, 3], \"dtype\": \"float32\", \"data_format\": \"channels_last\"}}, {\"class_name\": \"Dense\", \"config\": {\"name\": \"dense_6\", \"trainable\": true, \"dtype\": \"float32\", \"units\": 128, \"activation\": \"relu\", \"use_bias\": true, \"kernel_initializer\": {\"class_name\": \"GlorotUniform\", \"config\": {\"seed\": null}}, \"bias_initializer\": {\"class_name\": \"Zeros\", \"config\": {}}, \"kernel_regularizer\": null, \"bias_regularizer\": null, \"activity_regularizer\": null, \"kernel_constraint\": null, \"bias_constraint\": null}}, {\"class_name\": \"Dense\", \"config\": {\"name\": \"de

In [57]:
with open("2021-09-10-keras.json", "r") as my_file_read:
    my_list = json.load(my_file_read)

print(my_list)

{"class_name": "Sequential", "config": {"name": "sequential_3", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 200, 200, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "flatten_3_input"}}, {"class_name": "Flatten", "config": {"name": "flatten_3", "trainable": true, "batch_input_shape": [null, 200, 200, 3], "dtype": "float32", "data_format": "channels_last"}}, {"class_name": "Dense", "config": {"name": "dense_6", "trainable": true, "dtype": "float32", "units": 128, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_7", "trainable": true, "dtype": "float32", "units": 2, "activation": "linear", "use_bias": true, "kernel_initial